In [1]:
!pip install -U langchain langchain-community langchain-huggingface transformers accelerate
!pip install -q datasets sentencepiece gradio torch peft huggingface_hub
!pip install transformers datasets sentencepiece torch accelerate gradio --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.2
    Uninstalling transformers-4.55.2:
      Successfully uninstalled transformers-4.55.2
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.10.0
    Uninstalling accelerate-1.10.0:
      Successfully uninstalled accelerate-1.10.0


In [2]:
!pip install transformers gradio torch --quiet

In [3]:
from transformers import pipeline
import gradio as gr


In [ ]:
import os, json, random
import torch
from pathlib import Path
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments, pipeline
)
from datasets import load_dataset

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


Using device: cuda


In [ ]:
DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

# Base examples
base_pairs = [
    {"question": "'राम' शब्दस्य प्रथमा एकवचनं किम्?", "answer": "रामः"},
    {"question": "'रामम्' इति का विभक्तिः?", "answer": "द्वितीया विभक्तिः"},
    {"question": "सीता का लिङ्गः?", "answer": "स्त्रीलिङ्गः"},
    {"question": "What is 'vibhakti' in Sanskrit grammar?", "answer": "विभक्तिः = case inflection (7 cases)."},
    {"question": "गजस्य बहुवचन प्रथमा किम्?", "answer": "गजाः"},
    {"question": "What is sandhi (सन्धि)?", "answer": "सन्धिः = euphonic combination."},
]

# Add a few more grammar exercises
grammar_exercises = [
    {"question": "रामस्य सम्बोधन किम्?", "answer": "हे राम!"},
    {"question": "Translate: 'The boy goes to school' in Sanskrit.", "answer": "बालकः विद्यालयं गच्छति।"},
    {"question": "What is a dhātu (धातु)?", "answer": "धातुः = verb root, e.g. √गम् = to go."},
    {"question": "What is समास?", "answer": "समासः = compound word formation, e.g. राजपुत्रः."},
]

augmented = base_pairs + grammar_exercises

# Expand to ~200 examples by random sampling
while len(augmented) < 200:
    augmented.append(random.choice(augmented))

random.shuffle(augmented)

# Split train/valid
split_idx = int(0.85 * len(augmented))
train_data, valid_data = augmented[:split_idx], augmented[split_idx:]

def save_jsonl(data_list, filepath):
    with open(filepath, "w", encoding="utf-8") as f:
        for item in data_list:
            json_line = {"input": item["question"], "output": item["answer"]}
            f.write(json.dumps(json_line, ensure_ascii=False) + "\n")

save_jsonl(train_data, DATA_DIR / "train.jsonl")
save_jsonl(valid_data, DATA_DIR / "valid.jsonl")

print(f"Saved {len(train_data)} training and {len(valid_data)} validation examples")


Saved 170 training and 30 validation examples


In [ ]:
data_files = {"train": "data/train.jsonl", "validation": "data/valid.jsonl"}
datasets = load_dataset("json", data_files=data_files)
print(datasets)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 170
    })
    validation: Dataset({
        features: ['input', 'output'],
        num_rows: 30
    })
})


In [ ]:
model_name = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

max_input_length = 64
max_target_length = 64

def preprocess(batch):
    model_inputs = tokenizer(batch["input"], max_length=max_input_length,
                             truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch["output"], max_length=max_target_length,
                           truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = datasets.map(preprocess, batched=True, remove_columns=["input", "output"])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/170 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="mt5-sanskrit-chatbot",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=5,   # Increased epochs for better training
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=20,
    push_to_hub=False
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)


/tmp/ipython-input-1219079253.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,50.406400,42.484966
2,47.656700,36.942089
3,45.512300,34.811604
4,45.330100,33.778103
5,44.422700,33.412430


TrainOutput(global_step=110, training_loss=46.427593716708095, metrics={'train_runtime': 92.1085, 'train_samples_per_second': 9.228, 'train_steps_per_second': 1.194, 'total_flos': 56179654656000.0, 'train_loss': 46.427593716708095, 'epoch': 5.0})

In [ ]:
trainer.save_model("sanskrit_finetuned_mt5")
tokenizer.save_pretrained("sanskrit_finetuned_mt5")

('sanskrit_finetuned_mt5/tokenizer_config.json',
 'sanskrit_finetuned_mt5/special_tokens_map.json',
 'sanskrit_finetuned_mt5/spiece.model',
 'sanskrit_finetuned_mt5/added_tokens.json',
 'sanskrit_finetuned_mt5/tokenizer.json')

In [ ]:
# Cell 11: Inference

from transformers import pipeline

# Local model path (make sure this folder exists after training)
model_path = "./sanskrit_finetuned_mt5"

# Load pipeline from local directory
generator = pipeline(
    "text2text-generation",
    model=model_path,
    tokenizer=model_path,
    device=0  # use GPU
)

# Example query
query = "राम शब्दस्य प्रथमा एकवचनं किम्?"

# Generate answer
output = generator(
    query,
    max_new_tokens=50,
    num_beams=4,
    early_stopping=True
)

print("Input:", query)
print("Output:", output[0]["generated_text"])

Device set to use cuda:0


Input: राम शब्दस्य प्रथमा एकवचनं किम्?
Output: <extra_id_0>


In [4]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("# 📖 Sanskrit Grammar Chatbot (Fine-tuned)")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Ask me in Sanskrit or English...")
    clear = gr.Button("Clear")

    def respond(message, chat_history):
        bot_reply = ask_bot(message)
        chat_history.append((message, bot_reply))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()
